In [1]:
import pandas as pd
import numpy as np

In [53]:
# data_dir = './horses/'
# horses_df = pd.read_csv(data_dir + 'horses.csv',
#                         usecols=['age', 'sex_id', 'prize_money'])
# horse_sexes_df = pd.read_csv(data_dir + 'horse_sexes.csv').set_index('id')
# riders_df = pd.read_csv(data_dir + 'riders.csv').set_index('id')

In [182]:
# Dataset Source: https://www.kaggle.com/gunner38/horseracing
tips_df = pd.read_csv('horse_tips.csv', encoding='latin1')
tips_df['Bet Type'] = tips_df['Bet Type'].apply(lambda x : 1 if x == 'Win' else 0)
tips_df['Result'] = tips_df['Result'].apply(lambda x : 1 if x == 'Win' else 0)

horses = np.sort(tips_df['Horse'].unique())
tracks = np.sort(tips_df['Track'].unique())

tips_df['Horse'] = tips_df['Horse'].apply(lambda x : np.where(horses == x)[0][0])
tips_df['Track'] = tips_df['Track'].apply(lambda x : np.where(tracks == x)[0][0])

In [97]:
tipsters = tips_df['Tipster'].unique()

In [151]:
success_rates = dict()
for tipster in tipsters:
    successes = tips_df[(tips_df['Tipster'] == tipster) & (tips_df['Bet Type'] == tips_df['Result'])].shape[0]
    total = tips_df[(tips_df['Tipster'] == tipster)].shape[0]
    success_rates[tipster] = successes/total
successes_df = pd.DataFrame(pd.Series(success_rates), columns=['Success Rate']).sort_values(by='Success Rate', ascending=False)
successes_df.head(5)

,Success Rate
Tipster G,0.672414
Tipster C,0.616226
Tipster F,0.520548
Tipster M,0.520376
Tipster N,0.496386


In [149]:
X_G = tips_df[tips_df.Tipster=='Tipster G']
X_C = tips_df[tips_df.Tipster=='Tipster C']
X_F = tips_df[tips_df.Tipster=='Tipster F']
X_M = tips_df[tips_df.Tipster=='Tipster M']
X_N = tips_df[tips_df.Tipster=='Tipster N']

In [150]:
print([X_G.shape[0], X_C.shape[0], X_F.shape[0], X_M.shape[0], X_N.shape[0]])

[290, 1738, 146, 957, 415]


We model the online learning algorithm as follows.

In each round, we are given a betting scheme from each tipster. We model the quality function (the return) using OLS on the features. The true return is computed as if a single dollar were bet in any scheme from the odds and the true result.

In [183]:
tips_df

,UID,ID,Tipster,Date,Track,Horse,Bet Type,Odds,Result,TipsterActive
0,1,1,Tipster A,24/07/2015,2,5158,1,8.00,0,True
1,2,2,Tipster A,24/07/2015,96,13108,1,4.50,0,True
2,3,3,Tipster A,24/07/2015,114,13411,1,7.00,0,True
3,4,4,Tipster A,24/07/2015,74,8976,1,5.00,0,True
4,5,5,Tipster A,25/07/2015,2,10554,1,4.33,1,True
5,6,6,Tipster A,25/07/2015,114,327,1,6.00,0,True
6,7,7,Tipster A,25/07/2015,73,9791,1,6.00,0,True
7,8,8,Tipster A,25/07/2015,62,2019,1,6.00,0,True
8,9,9,Tipster A,26/07/2015,12,12101,1,5.50,0,True
9,10,10,Tipster A,26/07/2015,80,115,1,2.00,0,True
